In [1]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.linear_model import LogisticRegression
import openai
import torch



In [13]:
import re

In [3]:
import asyncio
import nest_asyncio
nest_asyncio.apply()  # 允许嵌套运行 event loop（用于 notebook/Colab）

In [2]:

# Load Financial PhraseBank dataset
dataset = load_dataset("financial_phrasebank", "sentences_allagree", cache_dir="./hf_cache")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

In [4]:
import pandas as pd

df = pd.read_csv("Sentences_75Agree.txt", sep="@", names=["sentence", "label"], engine="python",encoding='latin1')


In [5]:
# Set your OpenAI API key
openai.api_key = 'sk-tbTNYIOx8ITEYP88QD8OYiCqKrxkiHN8qvgiTw5fy5ZU9T24'


In [6]:
label_map = {"positive": 2, "neutral": 1, "negative": 0}
df['label'] = df['label'].map(label_map)


In [7]:
dataset=df
# Prepare data
texts = dataset['sentence']
labels = dataset['label']


In [38]:
# ✅ 划分训练测试集
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.05, random_state=42)

# ✅ 定义 GPT 分类器函数（稳定输出）
async def get_gpt_sentiment(text):
    client = openai.AsyncOpenAI(base_url="https://use.52apikey.cn/v1",api_key=openai.api_key)

    system_prompt = (
        "You are a sentiment classifier for financial texts. "
        "Classify each input into: 0 = negative, 1 = neutral, 2 = positive. "
        "Respond ONLY with one digit (0, 1, or 2). No explanation."
    )

    user_prompt = (
        "Text: 'The company reported a significant drop in earnings.'\nAnswer: 0\n\n"
        "Text: 'The revenue remained stable over the quarter.'\nAnswer: 1\n\n"
        "Text: 'The firm exceeded its growth expectations.'\nAnswer: 2\n\n"
        f"Text: '{text}'\nAnswer:"
    )

    try:
        response = await client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0
        )
        raw = response.choices[0].message.content.strip()
        r=re.findall(r'\b[012]\b', raw)
        return r[0]
    except Exception as e:
        print(f"⚠️ GPT 出错: {e}")
        return -1  # 占位错误标签

# ✅ GPT 预测（逐条调用）
loop = asyncio.get_event_loop()
gpt_predictions = []
print("🔍 GPT 开始预测...")

for i, text in enumerate(X_test):
    pred = loop.run_until_complete(get_gpt_sentiment(text))
    gpt_predictions.append(pred)
    if i % 10 == 0:
        print(f"{i+1}/{len(X_test)} finished...")

# ✅ 计算 GPT 性能
gpt_accuracy = accuracy_score(y_test, gpt_predictions)
gpt_f1 = f1_score(y_test, gpt_predictions, average='weighted')

print("\n🧠 ChatGPT-3.5-Turbo:")
print(f"Accuracy: {gpt_accuracy:.4f}")
print(f"F1 Score: {gpt_f1:.4f}")



🔍 GPT 开始预测...
1/173 finished...
11/173 finished...
21/173 finished...
31/173 finished...
41/173 finished...
51/173 finished...
61/173 finished...
71/173 finished...
81/173 finished...
91/173 finished...
101/173 finished...
111/173 finished...
121/173 finished...
131/173 finished...
141/173 finished...
151/173 finished...
161/173 finished...
171/173 finished...


ValueError: Mix of label input types (string and number)

In [39]:
# ➤ 强制类型一致
gpt_predictions_1 = list(map(int, gpt_predictions))


In [41]:
# ➤ 筛掉预测为 -1 的样本
valid_indices = [i for i, pred in enumerate(gpt_predictions_1) if pred != -1]
filtered_preds = [gpt_predictions_1[i] for i in valid_indices]
filtered_labels = [list(y_test)[i] for i in valid_indices]

# ➤ 重新计算分数
gpt_accuracy = accuracy_score(filtered_labels, filtered_preds)
gpt_f1 = f1_score(filtered_labels, filtered_preds, average='weighted')

In [42]:
print("\n🧠 ChatGPT-3.5-Turbo:")
print(f"Accuracy: {gpt_accuracy:.4f}")
print(f"F1 Score: {gpt_f1:.4f}")



🧠 ChatGPT-3.5-Turbo:
Accuracy: 0.9827
F1 Score: 0.9827


In [43]:
# ✅ 划分训练测试集X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.05, random_state=42)
# ✅ BERT（FinBERT）预测
print("🤖 加载 FinBERT 模型...")
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

bert_predictions = []
print("🔍 FinBERT 开始预测...")

for text in X_test:
    result = classifier(text)[0]
    sentiment_map = {'negative': 0, 'neutral': 1, 'positive': 2}
    pred = sentiment_map[result['label']]
    bert_predictions.append(pred)

bert_accuracy = accuracy_score(y_test, bert_predictions)
bert_f1 = f1_score(y_test, bert_predictions, average='weighted')

print("\n🤖 FinBERT:")
print(f"Accuracy: {bert_accuracy:.4f}")
print(f"F1 Score: {bert_f1:.4f}")



🤖 加载 FinBERT 模型...


Device set to use cpu


🔍 FinBERT 开始预测...

🤖 FinBERT:
Accuracy: 0.9480
F1 Score: 0.9486


In [44]:
# ✅ LDA + LogisticRegression
print("📚 LDA 特征提取中...")
vectorizer = CountVectorizer(max_features=1000, stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

lda = LatentDirichletAllocation(n_components=3, random_state=42)
X_train_lda = lda.fit_transform(X_train_vec)
X_test_lda = lda.transform(X_test_vec)

clf = LogisticRegression(max_iter=1000, random_state=42)
clf.fit(X_train_lda, y_train)
lda_predictions = clf.predict(X_test_lda)

lda_accuracy = accuracy_score(y_test, lda_predictions)
lda_f1 = f1_score(y_test, lda_predictions, average='weighted')

print("\n📊 LDA + LR:")
print(f"Accuracy: {lda_accuracy:.4f}")
print(f"F1 Score: {lda_f1:.4f}")

# ✅ 汇总结果
results = pd.DataFrame({
    'Model': ['ChatGPT-3.5-Turbo', 'FinBERT', 'LDA+LogReg'],
    'Accuracy': [gpt_accuracy, bert_accuracy, lda_accuracy],
    'F1 Score': [gpt_f1, bert_f1, lda_f1]
})

print("\n✅ 模型比较:")
print(results)

📚 LDA 特征提取中...

📊 LDA + LR:
Accuracy: 0.6821
F1 Score: 0.6266

✅ 模型比较:
               Model  Accuracy  F1 Score
0  ChatGPT-3.5-Turbo  0.982659  0.982714
1            FinBERT  0.947977  0.948613
2         LDA+LogReg  0.682081  0.626599
